#Classificação de Cogumelos

## Carregar base de Dados

In [ ]:
# Importação dos pacotes
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_excel('mushrooms.xlsx')
data.head()

In [ ]:
data.describe()

##Análise exploratória

In [ ]:
# Balanceamento
graph = sns.countplot(data=data, x='class')
plt.title('Balanceamento')

for idx, bar in enumerate(graph.patches):
    height = bar.get_height()
    graph.text(x=bar.get_x() + bar.get_width()/2., y=height + 0.1, s=(data['class'].value_counts()).sort_values()[idx], ha="center")

plt.show()

In [ ]:
# Atributo odor
plt.figure(figsize=(20,8))
sns.countplot(data=data, x='odor', hue='class', palette='husl')
plt.show()

In [ ]:
# Atributo veil.color
plt.figure(figsize=(20,8))
sns.countplot(data=data, x='veil.color', hue='class', palette='Set2')
plt.show()

In [ ]:
# Atributo stalk.shape
plt.figure(figsize=(20,8))
sns.countplot(data=data, x='stalk.shape', hue='class', palette='Paired')
plt.show()

In [ ]:
edible = data[data['class'] == 'EDIBLE']
poisonous = data[data['class'] == 'POISONOUS']

plt.hist(edible['spore.print.color'], alpha=0.5, label='edible')
plt.hist(poisonous['spore.print.color'],  alpha=0.5, label='poisonous')
plt.xticks(rotation=45);
plt.legend()
plt.show()

In [ ]:
sns.scatterplot(data=data, x='odor', y='veil.color', hue='class')
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
sns.stripplot(data=data, x='odor', y='veil.color', hue='class', jitter=0.8, dodge=True, orient='h', alpha=0.4)

plt.legend(loc='lower right')
plt.show()

##Machine Learning

## Separar bases de treino e teste

In [ ]:
# semente para reprodução de resultados
seed=10000

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.loc[:,data.columns != 'class'], data['class'], test_size=0.2, stratify=data['class'], random_state=seed)

print('shape input de treino: ', X_train.shape)
print('shape input de teste: ', X_test.shape)
print('shape output de treino: ', y_train.shape)
print('shape output de teste: ', y_test.shape)

In [ ]:
# Verificar proporções. amostragem estratificada
data['class'].value_counts()

In [ ]:
#treino
print(0.8 * data['class'].value_counts()[0])
print(0.8 * data['class'].value_counts()[1])

#test
print(0.2 * data['class'].value_counts()[0])
print(0.2 * data['class'].value_counts()[1])

In [ ]:
y_train.value_counts()

In [ ]:
y_test.value_counts()

## Árvore de Decisão

In [ ]:
# Trasnformar inputs categóricos em numérico: dummy coding
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(drop='first')
encoder.fit(X_train)
X_train = encoder.transform(X_train)
X_test = encoder.transform(X_test)

In [ ]:
X_train.shape

In [ ]:
# treinar modelo
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(min_samples_leaf=5, random_state=seed) # tente mudar parâmetro para evitar overfitting
model.fit(X_train, y_train);

In [ ]:
# Visualização gráfica da árvore de decisão
from sklearn import tree
fig, ax = plt.subplots(figsize=(10, 10)) # Definir tamanho da imagem a ser gerada
tree.plot_tree(model, class_names=y_train.unique(), filled=True, rounded=True); # plota a árvore

In [ ]:
# OU
# Visualização gráfica da árvore de decisão
from sklearn.tree import export_graphviz
import pydotplus
from IPython.display import Image

dot_data = export_graphviz(model, #feature_names=list(pd.DataFrame(X_train).columns.values), 
                           class_names=['Comestível', 'Venenoso'], rounded=True, filled=True)
graph = pydotplus.graph_from_dot_data(dot_data)
Image(graph.create_png())

In [ ]:
# métrica de treino
model.score(X_train, y_train)

In [ ]:
# Avaliar modelo treinado na base de teste
model.score(X_test, y_test) # acurácia

In [ ]:
y_pred = model.predict(X_test) #inferência do teste

In [ ]:
import numpy as np
unique_elements, counts_elements = np.unique(y_test, return_counts=True)
print("Frequência:")
print(np.asarray((unique_elements, counts_elements)))

In [ ]:
# Matriz de confusão
from sklearn.metrics import confusion_matrix
confMatrix = confusion_matrix(y_pred, y_test)
pd.DataFrame(confMatrix, index=['Previsto comestível', 'Previsto venenoso'], columns=['Real comestível', 'Real venenoso'])

In [ ]:
ax = plt.subplot()
sns.heatmap(confMatrix, annot=True, fmt=".0f")
plt.xlabel('Real')
plt.ylabel('Previsto')
plt.title('Matriz de Confusão')

# Colocar os nomes
ax.xaxis.set_ticklabels(['Comestível', 'Venenoso']) 
ax.yaxis.set_ticklabels(['Comestível', 'Venenoso'])
plt.show()